# 02 Label apps based on the type of contract

This notebook filters the apps based on what type of contract they have:

- token_contract: ERC20 token contracts. Metrics are derived from token transfer event data (Dune).

- dapp_contract: Regular smart contracts. Metrics are derived from raw transaction data (Goldsky). 

- factory_contract: Contracts used to generate other contracts. Metrics pendings.

- trace_contract: Something else, but that has events in traces.

In [1]:
from google.cloud import bigquery
import os
import pandas as pd

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../../oso_gcp_credentials.json"
client = bigquery.Client()
PROJECT = 'opensource-observer'
CURRENT_CHAINS = ['Optimism', 'Base', 'Zora', 'Mode', 'Frax', 'Metal']

In [3]:
all_apps = pd.read_csv('data/apps/applications_reviewed.csv')

## Part 1. Token contracts

In [4]:
token_list = pd.read_csv('data/raw_metric_data/dune_token_list.csv', low_memory=False)

tokens = {
    'Base': [],
    'Optimism': [],    
    'Zora': []
}

for _,row in token_list.iterrows():
    chain = row['blockchain'].title()
    addresses = row['token_addresses']
    addresses = addresses[1:-1]
    for a in addresses.split(' '):
        if len(a) == 42:
            tokens[chain].append(a.lower())

In [5]:
contract_labels = {}
for _,app in all_apps.iterrows():
    
    chain = app['chain']
    if not isinstance(chain, str):
        continue
    if chain == 'All Superchain':
        continue
    if chain not in CURRENT_CHAINS:
        contract_labels.update({app['uuid']: chain.lower()})
        print(f"[*{chain}*]: {app['name']}")
        continue
        
    address = app['address']
    if tokens.get(chain):
        if address in tokens[chain]:
            contract_labels.update({app['uuid']: 'token_contract'})
            print(f"[{chain}]: {app['name']}")
            continue
    
    contract_labels.update({app['uuid']: 'unknown'})

[Base]: Aerodrome Finance
[Base]: baby Dragon
[Base]: Bario Entertainment System
[Base]: Based Bodega
[Base]: BasedChad
[Base]: $BDOOGS
[Base]: BetBase
[*Cyber*]: Bitmon 
[*Lisk*]: Bitsave Protocol
[Base]: BMX
[Base]: BMX
[Base]: Boregon Trail
[Base]: BUILD
[Base]: BUZZAR
[*SwanChain*]: Chainnode
[*Kroma*]: Clutch
[*Mint*]: Comet Protocol
[*Mint*]: coNFT
[*Redstone*]: Dark Forest Ares 
[Optimism]: Demo STG Project - Contract - Prelaunch
[Base]: doginme memecoin
[*Kroma*]: EFIHUB
[Optimism]: Glo Dollar
[Base]: Gloom
[Optimism]: Idle
[*Mint*]: ImaginAIryNFTs
[Optimism]: JEFE TOKEN
[*SwanChain*]: Lagrange
[Optimism]: Lido Finance
[Base]: Lizard
[*Orderly*]: LogX
[Optimism]: MagicalTux
[*Mint*]: Mint Forest
[*Mint*]: MintSwap
[*Kroma*]: motoDEX
[*SwanChain*]: Multichain.Storage
[*Mint*]: NERZO
[*Kroma*]: NFTFeed
[*Mint*]: OmniHub
[*Mint*]: OpenALCHI
[*Orderly*]: Orderly Network
[*Lisk*]: Owlto Finance
[*Kroma*]: Owlto Finance
[*Mint*]: Owlto Finance
[*SwanChain*]: Owlto Finance
[Base]: Pea

## Part 2. Dapp contracts

In [6]:
other_projects = [k for k,v in contract_labels.items() if v == 'unknown']
contract_addresses = all_apps[all_apps['uuid'].isin(other_projects)]['address'].unique()
contract_addresses_str = "'" + "','".join(contract_addresses) + "'"
len(other_projects)

280

In [7]:
query = f"""
select 
    to_address, chain
    from `{PROJECT}.static_data_sources.sunny_contract_lookup`
    where
        to_address in ({contract_addresses_str})
        and txns >= 30
"""
result = client.query(query)
dapp_contracts = result.to_dataframe()

In [8]:
for _,app in all_apps.iterrows():
    chain = app['chain']
    address = app['address']
    uuid = app['uuid']
    temp_ = dapp_contracts[(dapp_contracts['to_address']==address) & (dapp_contracts['chain']==chain)]
    if len(temp_) == 1:
        contract_labels[uuid] = 'dapp_contract'
        print(f"[{chain}]: {app['name']}")

[Base]: 0x
[Optimism]: AI Frames Creator
[Base]: AI
[Optimism]: Air3
[Base]: Air3 on Base
[Optimism]: Amazing Hunt
[Optimism]: Another World
[Base]: Aragon 
[Base]: Arcadia Finance
[Base]: Avantis 
[Optimism]: Balancer
[Mode]: Balancer
[Base]: Balancer
[Optimism]: Balancer
[Base]: Balancer
[Base]: Based Bits
[Base]: Basepaint
[Base]: BaseSwap 
[Optimism]: Beefy
[Optimism]: Beefy
[Base]: Beefy
[Mode]: Beefy
[Base]: Beefy
[Base]: BerryLab
[Optimism]: BLOBz
[Base]: BLOBz on Base
[Mode]: BLOBz on Mode
[Zora]: BLOBz on Zora
[Base]: BLOCKLORDS
[Zora]: BLOCKS
[Base]: BMX by Morphex
[Base]: BMX
[Base]: BMX
[Base]: BMX
[Base]: BMX
[Base]: BMX
[Optimism]: Bored Town
[Base]: BSX
[Base]: Cat Town | Gacha
[Base]: Cat Town
[Base]: Cat Town
[Base]: CC: App Type 8.20.24
[Mode]: Charged Particles - Web3 Packs
[Base]: Checkin: Location-based “Checkin and Earn” social to connect IRL and onchain
[Base]: Chibi Clash Kingdoms
[Base]: Collectr
[Base]: Coop Records Music
[Optimism]: Copin.io
[Base]: CryptoRan

## Part 3. Factory contracts

In [9]:
other_projects = [k for k,v in contract_labels.items() if v == 'unknown']
other_addresses = all_apps[all_apps['uuid'].isin(other_projects)]['address'].unique()
other_addresses_str = "'" + "','".join(other_addresses) + "'"
len(other_projects)

90

In [12]:
query = f"""
select 
    factory_address, chain
    from `{PROJECT}.static_data_sources.sunny_factory_lookup`
    where factory_address in ({other_addresses_str})
"""
result = client.query(query)
factory_contracts = result.to_dataframe()
factory_contracts['chain'] = factory_contracts['chain'].str.title()

In [13]:
for _,app in all_apps.iterrows():
    chain = app['chain']
    address = app['address']
    uuid = app['uuid']
    temp_ = factory_contracts[(factory_contracts['factory_address']==address) & (factory_contracts['chain']==chain)]
    if len(temp_) == 1:
        contract_labels[uuid] = 'factory_contract'
        print(f"[{chain}]: {app['name']}")

[Base]: 5050IO
[Base]: AlfaFrens
[Base]: Automod
[Base]: Bond
[Base]: Crypto: The Game S2
[Base]: Crypto: The Game S2
[Base]: Flat Money
[Base]: Glass
[Base]: HALP
[Mode]: Kim Exchange
[Optimism]: Light
[Base]: LottoPGF
[Base]: NFTs2Me
[Optimism]: Pairwise
[Base]: Party.app + Party Protocol
[Optimism]: Sushiswap
[Base]: Vault


## Part 4. Trace contracts

In [14]:
other_projects = [k for k,v in contract_labels.items() if v == 'unknown']
trace_addresses = all_apps[all_apps['uuid'].isin(other_projects)]['address'].unique()
trace_addresses_str = "'" + "','".join(trace_addresses) + "'"
len(other_projects)

73

In [15]:
query = f"""
select 
    *
    from `{PROJECT}.static_data_sources.sunny_trace_lookup`
    where address in ({trace_addresses_str})
"""
result = client.query(query)
trace_contracts = result.to_dataframe()

In [16]:
for _,app in all_apps.iterrows():
    chain = app['chain']
    address = app['address']
    uuid = app['uuid']
    temp_ = trace_contracts[(trace_contracts['address']==address) & (trace_contracts['chain']==chain)]
    if len(temp_) >= 1:
        contract_labels[uuid] = 'trace_contract'
        print(f"[{chain}]: {app['name']}")

[Base]: Academy
[Optimism]: ArbiLearn
[Optimism]: ArbiLearn
[Base]: ArtGridz
[Base]: Attesta_Mesta
[Optimism]: BCard
[Base]: Definitive Finance
[Optimism]: Degen Deals
[Base]: Degenfans gated frames
[Base]: Dragon Academy
[Base]: DUDEGEN
[Base]: Feedback Forge - Decentralized Feedback Platform
[Base]: HAM
[Base]: KRASH WORLD: DA GAME PASS
[Optimism]: Metronome
[Mode]: ModeMax
[Base]: Nouns Boy
[Base]: nounspace
[Base]: Onchain Mecha
[Optimism]: OP Land
[Base]: Privy
[Base]: Recipes Onchain
[Optimism]: Scry
[Base]: Seamless Protocol
[Base]: /send
[Base]: /send
[Optimism]: Silo Finance
[Optimism]: Silo Finance
[Optimism]: Silo Finance
[Optimism]: Silo Finance
[Optimism]: Silo Finance
[Base]: Snickerdoodle
[Base]: song.camp: mint your music
[Base]: Spotlight
[Base]: SuperchainSynergyHub
[Base]: Talent Protocol
[Base]: Team Super Onchain 
[Base]: TOCD Studio
[Base]: Top Test
[Base]: Trash NFTs
[Optimism]: Travenesia
[Base]: Uniswap Position
[Optimism]: Vesper Finance
[Base]: Votecaster


## Part 5. Check what's left and dump it

In [17]:
other_projects = [k for k,v in contract_labels.items() if v == 'unknown']
len(other_projects)

29

In [18]:
project_apps = pd.read_csv('data/apps/project_apps.csv', index_col=0)
project_apps['contract_type'] = project_apps['uuid'].map(contract_labels)

In [19]:
project_apps.to_csv('data/apps/project_apps_labeled.csv')